In [5]:
import os,shutil
from PIL import Image
import numpy as np

In [23]:
cur_dir = os.getcwd()

dataset_dir = os.path.join(cur_dir,"Dataset")

labels_dir = os.path.join(dataset_dir,"labels")
frames_dir = os.path.join(dataset_dir,"frames")

n_lecs = len(os.listdir(labels_dir))

listOfLectures = []

for lec in os.listdir(frames_dir):
    frame_list = []
    lec_path = os.path.join(frames_dir,lec)
    for frame in os.listdir(lec_path):
        frame_list.append([os.path.join(lec_path,frame),])
    listOfLectures.append(frame_list)

# print(listOfLectures[:2])

In [22]:
lec_num = 0
for csv in os.listdir(labels_dir):
    csv_path = os.path.join(labels_dir,csv)
    with open(csv_path,'r') as input:
        data = input.read()
        data = data.split()
        for i in range(len(data)):
            listOfLectures[lec_num][i].append(int(data[i]))
    lec_num+=1

In [ ]:
# listOfLectures : list of lectures where each lecture contains a list of form [frame_number, whether hit or miss ]

